In [11]:
import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import h5py
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
vmec_file = "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias4_vmec.nc4"

from jax_sbgeom.flux_surfaces.flux_surfaces_base import _cartesian_position_interpolated_jit, _cylindrical_position_interpolated, _cartesian_position_interpolated_grad, ToroidalExtent
from tests.flux_surfaces.flux_surface_base import test_position, _get_flux_surfaces, _sampling_grid, _1d_sampling_grid, test_normals, test_meshing_surface, test_principal_curvatures


test_pos          = False
test_norm         = False
test_meshing_surf = False
test_principal_curv  = False
if test_pos:
    test_position(vmec_file, n_repetitions=10)

if test_norm:
    test_normals(vmec_file, n_repetitions=10)

if test_meshing_surf:
    test_meshing_surface(vmec_file, n_repetitions=10)


if test_principal_curv:
    test_principal_curvatures(vmec_file, n_repetitions=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def setup_timing_grid(vmec_file, ns, nt, np, include_axis=False):
    fs_jax, fs_sbgeom = _get_flux_surfaces(vmec_file)
    s,t, p = _sampling_grid(fs_jax, ns, nt, np,  include_axis=include_axis)

    return fs_jax, (s,t,p), fs_sbgeom


fs_jax, (s,t,p), fs_sbgeom = setup_timing_grid(vmec_file, 50, 50, 50, include_axis=False)

In [8]:
from jax_sbgeom.jax_utils import stack_jacfwd
from jax_sbgeom.flux_surfaces.flux_surfaces_base import _cartesian_position_interpolated_grad_grad, _principal_curvatures_interpolated




In [9]:
_cartesian_position_interpolated_grad_grad(fs_jax.data, fs_jax.settings, 1.0, jnp.linspace(0,1,10), jnp.linspace(0,2*jnp.pi,10)).shape

(10, 3, 2, 2)

In [36]:
_principal_curvatures_interpolated(fs_jax.data, fs_jax.settings, 1.0, 0.5,jnp.array(0.3, ))

(3,) ()


Array([0.08616126, 0.32103396], dtype=float64)

In [29]:
fs_sbgeom.Return_Principal_Curvatures(1.0, 0.0, 0.5, 0.3)

array([0.08616126, 0.32103396])